# Host's ganglia transcriptome assembly
In this notebook you can find the commands and scripts that were used for the transcriptome assembly.
Input and output directories are specified under `### input ###` in every script. The path to the executable file of the program is specified in the `### soft ###` section of the script. More datails are noted in comments in every script.

## Library preparation
### Data reformatting
Here are scripts for analyzing female samples. For male samples directories paths should be changed.

BBMap/BBTools (v. 39.01) was used for ID correction of short paired-end reads and filtering of rnaSPAdes results (removal of assembled sequences with unknown nucleotides).

Script for BBTools:

In [ ]:
# !/bin/bash

### INPUT ###

INDIR=/home/al/Documents/projects/reticulata_data/females/input/ # Full path to the input directory
OUTDIR=/home/al/Documents/projects/reticulata_data/females/output/ # Full path to the output directory
SPTAG=PmPr_fem # Species tag
OUTTAG=renamed_raw # Output files tag

### SOFT ###

BBTools_reformat=/home/al/Documents/Programs/BBMap_39.01/bbmap/reformat.sh

### MAIN ###

for dir in $(find $INDIR -mindepth 1 -type d); do
    r1="$(find $dir -type f -name '*_R1.fq.gz')"
    r2="$(find $dir -type f -name '*_R2.fq.gz')"
    tag=$(basename $dir)
    echo "BBtools/reformat.sh starts to work with: "
    echo "R1: $r1"
    echo "R2: $r2"
    echo "TAG: $tag"
    mkdir $OUTDIR/$tag
    cd $OUTDIR/$tag
    nohup $BBTools_reformat in=$r1 in2=$r2 out=${SPTAG}.${tag}.${OUTTAG}.R1.fq.gz out2=${SPTAG}.${tag}.${OUTTAG}.R2.fq.gz trimreaddescription=t addslash=t spaceslash=f
    wait
    cd ..
done

## Short paired-end reads libraries quality control and preparation

FastQC (v. 0.12.1) was utilized to spot potential problems in sequencing datasets.

FastQC launching command example:

In [ ]:
fastqc -o infe_fem_fq_out *.fq.gz

FastP utility (v. 0.23.2) performed the low-quality and adapter sequences removal. Scripts are available in the repository. The trimming parameters was the following: `--cut_window_size 4 --cut_mean_quality 20 --qualified_quality_phred 20 --length_required 25`.

FastP utility launching script:

In [ ]:
#!/bin/bash

### INPUT ###

INDIR=/home/LVP/raw_data/rna/females/for_fastp # Full path to the input directory
OUTDIR=/home/LVP/raw_data/rna/females/fastp_out # Full path to the output directory

### Soft ###

fastp=/home/LVP/Soft/fastp

### PARAMS ###

WIN_SIZE=4
QUAL_QUAL=20
MEAN_QUAL=20
MIN_LEN=25
THREADS=5

### MAIN ###

cd $OUTDIR

for dir in $(find $INDIR -mindepth 1 -type d); do
    r1="$(find $dir -type f -name '*R1.fq.gz')"
    r2="$(find $dir -type f -name '*R2.fq.gz')"
    tag=$(basename $dir)
    echo "FastP starts to work with: "
    echo "R1: $r1"
    echo "R2: $r2"
    echo "TAG: $tag"
    echo "Window size: $WIN_SIZE"
    echo "Mean quality: $MEAN_QUAL"
    echo "Qualified quality: $QUAL_QUAL"
    echo "Min length: $MIN_LEN"
    echo "Threads: $THREADS"
    mkdir ${tag}_fastP_output
    cd ${tag}_fastP_output
    nohup $fastp --in1 $r1 --out1 ${tag}.fastp_tmm.R1.fastq.gz --in2 $r2 --out2 ${tag}.fastp_tmm.R2.fastq.gz --unpaired1 ${tag}.fastp_unpaired1.fastq.gz --unpaired2 ${tag}.fastp_unpaired2.fastq.gz$
    wait
    cd ..
done

echo "##### Job is complete #####"


## Decontamination via Kraken2
Kraken2 (v. 2.1.2) is a taxonomic classification system using exact k-mer matches to achieve high accuracy and fast classification speeds. In our project, we used it to build a database of possible contamination (such as bacteria and viruses). We used standard database that includes libraries from archaea, viruses, bacteria, plasmid, fungi, human and protozoa. 
In case of transcriptomic short paired-end read libraries we added to standard database the early assembled transcriptomic data from the parasite *Peltogaster reticulata* ([Nesterenko, Miroliubov, 2023](https://f1000research.com/articles/11-583)) and the genomic sequences from another rhizocephalan species, *Sacculina carcini* ([Blaxter et al., *in press*](https://wellcomeopenresearch.org/articles/8-91)). 

Standard database downloading:

In [ ]:
#!/bin/bash

### INPUT ###

OUTDIR=/home/LVP/Kraken2_DB_reticulata/ # Full path to the directory where the database will be stored
DBTAG=Kraken2_plus_db_reticulata # Some name of the database to be created

### SOFT ###

KRAKEN2DIR=/home/LVP/Soft/kraken2 # Full path to the directory where the Kraken2 executable files are stored

### Processing ###

mkdir -p $OUTDIR
cd $OUTDIR

nohup $KRAKEN2DIR/kraken2-build --download-taxonomy --db $DBTAG --threads 5 --use-ftp
wait
nohup $KRAKEN2DIR/kraken2-build --download-library archaea --db $DBTAG --threads 5
wait
nohup $KRAKEN2DIR/kraken2-build --download-library viral --db $DBTAG --threads 5
wait
nohup $KRAKEN2DIR/kraken2-build --download-library bacteria --db $DBTAG --threads 5
wait
nohup $KRAKEN2DIR/kraken2-build --download-library plasmid --db $DBTAG --threads 5
wait
nohup $KRAKEN2DIR/kraken2-build --download-library human --db $DBTAG --threads 5
wait
nohup $KRAKEN2DIR/kraken2-build --download-library fungi --db $DBTAG --threads 5
wait
nohup $KRAKEN2DIR/kraken2-build --download-library protozoa --db $DBTAG --threads 5
wait
nohup $KRAKEN2DIR/kraken2-build --download-library UniVec_Core --db $DBTAG --threads 5
wait

echo "##### Job is complete #####"

*Sacculina carcini* genome was added to standard database using the command:

In [ ]:
/home/LVP/Soft/kraken2/kraken2-build --add-to-library /home/LVP/Source/Sacculina/Sacculinacarcini_ref_genes.with_taxid.fasta --db /home/LVP/kraken2_db_females/Kraken2_plus_db_reticulata --threads 10

Build database:

In [ ]:
# !/bin/bash

### INPUT ###

OUTDIR=/home/LVP/kraken2_fem # Full path to the directory where the database will be stored
DBTAG=/home/LVP/kraken2_db_females/Kraken2_plus_db_reticulata # Some name of the database to be created

### SOFT ###

KRAKEN2DIR=/home/LVP/Soft/kraken2 # Full path to the directory where the Kraken2 executable files are stored

### Processing ###

cd $OUTDIR

nohup $KRAKEN2DIR/kraken2-build --build --db $DBTAG --threads 12
wait

echo "##### Job is complete #####"

Decontamination:

In [ ]:
#!/bin/bash

### Input ###

FASTPDIR=/home/LVP/fastp/rna/females/ # Full path to the directory where the fastP results are stored
OUTDIR=/home/LVP/kraken2_outputs/females_kraken2_output/ # Full path to the directory where the classification results (Kraken2 output) will be stored
DBDIR=/home/LVP/kraken2_db_females # Full path to the directory where the database is stored
DBTAG=Kraken2_plus_db_reticulata # Some name of the database that will be used
SUFFIX=#

### SOFT ###

KRAKEN2DIR=/home/LVP/Soft/kraken2 # Full path to the directory where the Kraken2 executable files are stored

### Processing ###

mkdir -p $OUTDIR
cd $OUTDIR

for dir in $(find $FASTPDIR -mindepth 1 -type d); do
        r1="$(find $dir -type f -name '*fastp_tmm.R1.fastq.gz')"
        r2="$(find $dir -type f -name '*fastp_tmm.R2.fastq.gz')"
        tag=$(basename $dir)
        echo "Kraken2 starts to work with: "
        echo "R1: $r1"
        echo "R2: $r2"
        echo "TAG: $tag"
      	mkdir ${tag}_vs_${DBTAG}
      	cd ${tag}_vs_${DBTAG}
        nohup $KRAKEN2DIR/kraken2 --db ${DBDIR}/${DBTAG}/ --threads 12 --paired --gzip-compressed --unclassified-out ${tag}.unclass.R${SUFFIX}.fastq --classified-out ${tag}.class.R${SUFFIX}.fastq --output ${tag}_vs_${DBTAG}.tab --report ${tag}_vs_${DBTAG}.report ${r1} ${r2}
	      cd ..
        wait
done

echo "##### Job is complete #####"

The results were visualized by Pavian (v. 1.0) using Kraken2 output files `.report` for every sample.

## Assembly
We performed *de novo* transcriptome assembly using 3 tools (rnaSPAdes, RNA-Bloom, and Trinity) in order to receive the best results using the strengths of different assemblers.

**Before assemblying we merged all prepared short paired-end read libraries from healthy and infected female and male crabs into the single fastq files.**


### rnaSPAdes

rnaSPAdes (SPAdes v. 3.15.4) with default options was used. The script is available below.

In [ ]:
#!/bin/bash

### INPUT ###
R1=/home/LVP/assembly/ganglia.merged.R1.fastq # Full path to the merged decontaminated (after Kraken2) R1 fastq file
R2=/home/LVP/assembly/ganglia.merged.R2.fastq # Full path to the merged decontaminated (after Kraken2) R2 fastq file
OUTDIR=/home/LVP/assembly/ganglia_ref_rnaSPAdes # Full path to the directory where the assembling output will be stored

### SOFT ###

SPADES=/home/LVP/Soft/SPAdes-3.15.4-Linux/bin/rnaspades.py

### MAIN ###
echo "***** rnaSPAdes began to work with params: *****"
echo "R1: $R1"
echo "R2: $R2"
echo "OUTDIR: $OUTDIR"
mkdir $OUTDIR
nohup $SPADES -1 $R1 -2 $R2 --ss fr --threads 24 --memory 100 -o $OUTDIR
wait
echo "##### Job is complete #####"


### RNA-Bloom
RNA-Bloom (v. 2.0.1) was installed using mamba (v. 1.4.1). Specified options was` --kmer 25` `--percent 0.90` `--length 200`. The script is the following.

In [ ]:
#!/bin/bash

### INPUT ###

R1=/home/LVP/assembly/ganglia.merged.R1.fastq  # Full path to the merged decontaminated (after Kraken2) R1 fastq file
R2=/home/LVP/assembly/ganglia.merged.R2.fastq  # Full path to the merged decontaminated (after Kraken2) R2 fastq file
OUTTAG=ganglia_ref_rnabloom # Some name of the assembly. For example, Pret_ref_rnabloom
OUTDIR=/home/LVP/assembly/ganglia_ref_rnabloom # Full path to the directory where the assembling output will be stored

### SOFT ###

RNABLOOM=/home/lianguzova/mambaforge/bin/rnabloom # Full path to the RNA-Bloom

### MAIN ###
echo "***** RNA-Bloom began to work with params: *****"
echo "R1: $R1"
echo "R2: $R2"
echo "OUTTAG: $OUTTAG"
echo "OUTDIR: $OUTDIR"

mkdir $OUTDIR
nohup $RNABLOOM --left $R1 --right $R2 --revcomp-right --name $OUTTAG --threads 24 --outdir $OUTDIR --kmer 25 --memory 100 --percent 0.90 -length 200
wait
echo "##### Job is complete #####"


### Trinity
Trinity (v. 2.14.0) was utilised. We also installed the tool dependencies using conda (v. 23.1.0) into special virtual environment. Here is the script that was used:

In [ ]:
#!/bin/bash

# INPUT: #
R1=/home/LVP/assembly/ganglia.merged.R1.fastq # Full path to the merged decontaminated (after Kraken2) R1 fastq file
R2=/home/LVP/assembly/ganglia.merged.R2.fastq # Full path to the merged decontaminated (after Kraken2) R2 fastq file
OUTTAG=trinity_ass # Some name of the assembly. For example, Pret_ref_Trinity
OUTDIR=/home/LVP/assembly/ganglia_ref_trinity_please  # Full path to the directory where the assembling output will be stored

### SOFT ###

TRINITY=/home/LVP/Soft/trinityrnaseq-v2.14.0/Trinity # Full path to the Trinity

### MAIN ###
echo "***** Trinity began to work with params: *****"
echo "R1: $R1"
echo "R2: $R2"
echo "OUTTAG: $OUTTAG"
echo "OUTDIR: $OUTDIR"

source activate /home/LVP/Soft/exit/envs/trinity_env # conda env

cd $OUTDIR

nohup $TRINITY --seqType fq --max_memory 100G --left $R1 --right $R2 --SS_lib_type FR --CPU 24 --min_contig_length 200 --output $OUTTAG --full_cleanup
wait
echo "##### Job is complete #####"

***In further analysis, we considered only sequences with a length of at least 200 nucleotides.*** To filter the assembly results, we used the python script to the all obtained transcriptomic assemblies:

In [ ]:
try:
    import argparse
except ImportError:
    print("Please check if module 'argparse' is installed")
    quit()

from Bio import SeqIO

parser = argparse.ArgumentParser()
parser.add_argument('--fasta', type=argparse.FileType('r'), required=True)
parser.add_argument('--min_len', type=str, required=True)
parser.add_argument('--out', type=str, required=True)
args = parser.parse_args()


if __name__ == "__main__":
    fasta_seqs = SeqIO.parse(args.fasta, "fasta")
    with open("{output}.fasta".format(output=args.out), 'a') as output:
        for fasta in fasta_seqs:
            name, sequence = fasta.id, fasta.seq
            if len(sequence) >= int(args.min_len):
                output.write(">{name}\n{seq}\n".format(name=name, seq=sequence))

## Clusterization

CD-HIT-est (v. 4.8.1) was used to the three transcriptome assemblies separate clusterization. Clusters with 95% identity were gathered, comparison of both strand (+/+, +/-) was performed. An example script is shown below:

In [ ]:
#!/bin/bash

# INPUT: #

FASTA=/home/LVP/assembly/ganglia_ref_trinity_please/filtred.fasta
TAG=Pdum_new_ref_trinity.cdhit_c95
OUTDIR=/home/LVP/assembly/trinity_cdhit_95_200

# SOFT: #

source activate /home/LVP/Soft/exit/envs/cdhit_env

# MAIN: #

cd $OUTDIR

echo "***** CDHIT began to work with params: *****"
echo "FASTA: $FASTA"
echo "TAG: $TAG"
echo "OUTDIR: $OUTDIR"
nohup /home/LVP/Soft/exit/envs/cdhit_env/bin/cd-hit-est -i $FASTA -o ${TAG}.fasta -c 0.95 -T 24 -d 0 -g 1 -r 1 -M 5000
wait
echo "##### Job is complete #####"

## Assembly quality control


TransRate (v. 1.0.1) was utilised for the qualitative analysis of *de novo* transcriptome assemblies.

The default options were used; the following scripts for Trinity output can be used as a template for other assemblers. 

**Only the sequences with high quality and assembly completeness (“good” according to the TransRate classification) were used in the following analysis.**

In [ ]:
#!/bin/bash

### INPUT ###

R1=/home/LVP/assembly/ganglia.merged.R1.fastq # Full path to the decontaminated and merged R1 fastq file
R2=/home/LVP/assembly/ganglia.merged.R2.fastq # Full path to the decontaminated and merged R2 fastq file
Contigs=/home/LVP/assembly/trinity_cdhit_95_200/Pdum_new_ref_trinity.cdhit_c95.fasta # Full path to the fasta file with assembled contigs (length >= 200 bp)
OUTDIR=/home/gafarova/trinity_TransRate_output # Full path to the directory where the quality control results will be stored

### SOFT ##

source activate /home/LVP/Soft/exit/envs/transrate_env # Full path to the TransRate virtual environment

TRANSRATE=/home/LVP/Soft/transrate-1.0.1/bin/transrate # Full path to TransRate software

### MAIN ###

echo "***** TransRate (v1.0.1) began to work with params: *****"
echo "R1: $R1"
echo "R2: $R2"
echo "Assembly: $Contigs"
cd $OUTDIR
nohup ${TRANSRATE} --assembly=$Contigs --left=$R1 --right=$R2 --threads=24 --output=${OUTDIR}
wait
echo "##### =^^= Job is complete #####"

**All the contigs with high rates of quality and assembly completeness were merged using basic UNIX-commands.**

## Clusterization

Transcriptome redundancy reduction was performed using CD-HIT clusterization with minimal 95% identity between contigs and both strand comparison. The script:

In [ ]:
#!/bin/bash

# INPUT: #

FASTA=/home/LVP/assembly/whole_assembly.fasta
TAG=whole_assembly_CDHIT
OUTDIR=/home/LVP/assembly/CDHIT_whole_cluster_95_percent

# SOFT: #

source activate /home/lianguzova/mambaforge/envs/cdhit_env

# MAIN: #

mkdir $OUTDIR
cd $OUTDIR

echo "***** CDHIT began to work with params: *****"
echo "FASTA: $FASTA"
echo "TAG: $TAG"
echo "OUTDIR: $OUTDIR"
nohup /home/lianguzova/mambaforge/envs/cdhit_env/bin/cd-hit-est -i $FASTA -o ${TAG}.fasta -c 0.95 -T 24 -d 0 -g 1 -r 1 -M 5000
wait
echo "##### Job is complete #####"

## Transcriptome assemblies completeness control

Benchmarking Universal Single-Copy Ortholog (BUSCO) (v. 5.4.7) was used for the transcriptome assemblies completeness control via the searching of metazoan single-copy orthologs. The transcriptome mode was selected (`-m transcriptome`), and Metazoa OrthoDB (v. 10) was used as lineage dataset. The template script is below. The Venn diagram visualization was created using [InteractiVenn](http://www.interactivenn.net/).

In [ ]:
#!/bin/bash

source activate busco_env

workdir=/home/LVP/busco/

cd $workdir

# INPUT: #
Contigs=/home/LVP/assembly/CDHIT_whole_cluster_95_percent/whole_assembly_CDHIT.fasta 
TAG=good_Pret_new_ref_whole_assembly_vs_Metazoa_odb10
DB=/home/LVP/Source/BUSCO/metazoa_odb10

# Analysis: #
echo "***** BUSCO began to work with params: *****"
echo "DB: $DB"
echo "TAG: $TAG"
echo "Assembly: $Contigs"
nohup /home/lianguzova/mambaforge/envs/busco_env/bin/busco -m transcriptome -i $Contigs -o $TAG -l $DB --cpu 12
wait
echo "##### Job is complete #####"

## Quantifying transcripts expression
Salmon (v. 1.10.1) installed in special conda environment mapped the short paired-end reads libraries to the reference transcriptome assembly.

The script allowing to create an index for the reference transcriptome:


In [ ]:
#!/bin/bash

workdir=/home/LVP/salmon_cleaned_assembly

cd $workdir

# INPUT: #
FASTA=/home/LVP/assembly/cleaned/cleaned_assembly.fasta
TAG=good_Pdum_new_ref_index

echo "***** Salmon (index) began to work with params: *****"
echo "FASTA: $FASTA"
echo "TAG: $TAG"
nohup salmon index --transcripts ${FASTA} --kmerLen 25 --index ${TAG} --threads 10
wait
echo "##### Job is complete #####"

Mapping of the paired-end reads and the sequences expression quantification was performed using the following script for the data from the female hermit crabs, which can be changed for the males.

In [ ]:
#!/bin/bash

### INPUT: ###
tmmdir=/home/LVP/kraken2_outputs/females_kraken2_output
outdir=/home/LVP/salmon_cleaned_assembly/
index=/home/LVP/salmon_cleaned_assembly/good_Pdum_new_ref_index

### Analyses ###

cd $outdir

for dir in $(find $tmmdir -mindepth 1 -type d); do
    r1="$(find $dir -type f -name '*class.R_1.fastq')" 
    r2="$(find $dir -type f -name '*class.R_2.fastq')"
    tag=$(basename -- $dir _vs_PlusPF_db)
    echo "Salmon quant starts to work with: "
    echo "R1: $r1"
    echo "R2: $r2"
    echo "TAG: $tag"
    echo "Ref_index: $index"
 nohup salmon quant -i ${index} -l A -1 $r1 -2 $r2 -o ${tag}_quant --seqBias --gcBias --minScoreFraction 0.50 --softclip --threads 12 --validateMappings --writeUnmappedNames
    wait
done

echo "##### Job is complete =^_^= #####"

## Determination of assembly translation products

TransDecoder (v. 5.5.0) identified candidate coding regions within transcript sequences. Initial identification of open reading frames (ORFs) and translation products was performed using following script:


In [ ]:
#!/bin/bash

source activate transdecoder_env

workdir=/home/LVP/transdecoder/cleaned/again

cd $workdir

# INPUT: #
FASTA=/home/LVP/assembly/CDHIT_cleaned_cluster_95_percent/cleaned_assembly_CDHIT.fasta  

# Analysis: #
echo "***** TransDecoder.LongORFs began to work with params: *****"
echo "FASTA: $FASTA"
nohup /home/lianguzova/mambaforge/envs/transdecoder_env/bin/TransDecoder.LongOrfs -t $FASTA -m 100 -G universal
wait
echo "##### Job is complete #####"

HMMER (v. 3.3.2) was used to compare found ORFs translation products with Pfam-A database using the following script. 

In [ ]:
#!/bin/bash

source activate transdecoder_env

workdir=/home/LVP/transdecoder/cleaned

# INPUT: #
FASTA=/home/LVP/transdecoder/cleaned/cleaned_assembly_CDHIT.fasta.transdecoder_dir/longest_orfs.pep
TAG=pfama_pret_cleaned

# Analysis: #
cd $workdir
echo "***** HMMsearch began to search (vs PfamA) with params: *****"
echo "FASTA: $FASTA"
echo "TAG: $TAG"
nohup /home/lianguzova/mambaforge/envs/transdecoder_env/bin/hmmsearch --cpu 12 --domtblout ${TAG}_longORFs_vs_pfam.domtblout /home/LVP/Source/Pfam-A.hmm $FASTA
wait
echo "##### Job is complete #####"

Comparison of amino acid sequences with the UniRef90 database was performed using the DIAMOND (v. 2.0.15) program. The script used is shown below:

In [ ]:
#!/bin/bash

source activate diamond_env

workdir=/home/LVP/transdecoder/cleaned

# INPUT: #
FASTA=/home/LVP/transdecoder/cleaned/cleaned_assembly_CDHIT.fasta.transdecoder_dir/longest_orfs.pep
DB=/home/LVP/transdecoder/uniref
TAG=Uniref_Pret_cleaned

# Analysis: #
echo "***** DIAMOND BLASTp began to search (vs UniRef90) with params: *****"
echo "FASTA: $FASTA"
echo "DB: $DB"
echo "TAG: $TAG"
nohup /home/lianguzova/mambaforge/envs/diamond_env/bin/diamond blastp --db $DB --query $FASTA --threads 12 --out ${workdir}/${TAG}_longest_orfs_vs_UniRef90.diamond.outfmt6 --outfmt 6 --max-target-seqs 1 --evalue 1e-5
wait
echo "##### Job is complete #####"

Refinement of ORFs translation products was performed using the results of comparison with databases:

In [ ]:
#!/bin/bash

source activate transdecoder_env

workdir=/home/LVP/transdecoder/cleaned/

cd $workdir

# INPUT: #
FASTA=/home/LVP/assembly/CDHIT_cleaned_cluster_95_percent/cleaned_assembly_CDHIT.fasta  
PFAM=/home/LVP/transdecoder/cleaned/pfama_pret_cleaned_longORFs_vs_pfam.domtblout
BLASTP=/home/LVP/transdecoder/cleaned/Uniref_Pret_cleaned_longest_orfs_vs_UniRef90.diamond.outfmt6

# Analysis: #
echo "***** TransDecoder.Predict began to work with params: *****"
echo "FASTA: $FASTA"
echo "PFAM: $PFAM"
echo "BLASTP: $BLASTP"
nohup /home/lianguzova/mambaforge/envs/transdecoder_env/bin/TransDecoder.Predict -t $FASTA --retain_pfam_hits $PFAM --retain_blastp_hits $BLASTP
wait
echo "##### Job is complete #####"

## Preparation of protein-coding sequences reference set

We selected transcripts encoding proteins consisting of more than 100 amino acids and with the expression level more than 2 transcripts per million (TPM) (According to the Wagner, Kin & Lynch (2013) publication, “genes with more than two transcripts per million transcripts (TPM) are highly likely from actively transcribed genes.”) at least in one sample analysed using the provided steps:

### Scripts for sorting transcripts and proteins `.fasta` files and expression levels `.csv` files

In [ ]:
import pandas as pd
from Bio import SeqIO
import re

### Select transcripts with a significant level of expression
Select from the expression-averaged table the identifiers of those sequences that have an expression level of at least 2 TPM in at least 1 sample.

In [ ]:
data = pd.read_csv('mean_expression_data.csv', delimiter=',')
data = data.query("healthy_fem>=2  or infected_fem>=2 or healthy_male>=2 or infected_male>=2")
data.to_csv('overexpressed.csv', index=False)

### Select long protein products
Select sequence identifiers that encode proteins at least 100 a.a. long

In [ ]:
df = pd.DataFrame(columns=['Name', 'Prot_name', 'Length'])
for record in SeqIO.parse("cleaned_assembly_proteins.fasta", "fasta"):
    name = re.split('\.p[\d]+$', record.id)[0]
    prot_num = record.id
    seq_len = len(record.seq)
    if seq_len >= 100:
        new_row = {'Name': name, 'Prot_name': prot_name, 'Length': seq_len}
        df = df._append(new_row, ignore_index=True)

df.to_csv('fasta_prot_id.csv', index=False)

### Merge identifiers that encode long proteins products and transcripts with significant expression levels

In [ ]:
def get_prot_num(x):
    y = re.search('\.p([\d]+)$', x)
    if y is None:
        return x
    else:
        return y.groups()[0]

data1 = pd.read_csv('fasta_prot_id.csv', delimiter=',')
data1['Prot_num'] = data1.apply(lambda row: get_prot_num(row['Prot_name']), axis=1)
names_list = data1.Name.unique()
print(data1.head())
data1 = data1.sort_values(by=['Prot_num'], kind='stable')
data1 = data1.sort_values(by=['Length'], kind='stable', ascending=False)
data1 = data1.sort_values(by=['Name'], kind='stable')
data1 = data1.reset_index(drop=True)
print(len(data1.Name.unique()))
data1 = data1[~data1.duplicated(subset=['Name'])]

data2 = pd.read_csv('overexpressed.csv', delimiter=',')

data = pd.merge(data1, data2)
data.to_csv('merged.csv', index=False)

### Make new expression tables

In [ ]:
data = pd.read_csv('merged.csv', delimiter=',')
ids = data.Name
new_names = pd.DataFrame({'New_name': ['Pmin_ref_seq_' + str(i+1) for i in range(len(ids))]})
new_names_dict = pd.concat([pd.DataFrame({'Name': ids}), new_names], axis=1)
print(new_names_dict.head(10))
print(len(new_names_dict))
table1 = pd.read_csv('expression_data.csv', delimiter=',')
table2 = pd.read_csv('mean_expression_data.csv', delimiter=',')
new_table1 = pd.merge(new_names_dict, table1, how='left')
new_table1 = new_table1.drop('Name', axis=1)
new_table1.rename(columns={'New_name': 'Name'}, inplace=True)
new_table1.to_csv('new_expression_data.csv', index=False)
new_table2 = pd.merge(new_names_dict, table2, how='left')
new_table2 = new_table2.drop('Name', axis=1)
new_table2.rename(columns={'New_name': 'Name'}, inplace=True)
new_table2.to_csv('new_mean_expression_data.csv', index=False)

### Make new `.fasta` protein file

In [ ]:
data = pd.read_csv('merged.csv', delimiter=',')
ids = data.Name
prot_ids = data.Prot_name
new_names = pd.DataFrame({'New_name': ['Pmin_ref_seq_' + str(i+1) for i in range(len(data))]})
new_names_dict = pd.concat([pd.DataFrame({'Name': ids}), pd.DataFrame({'Prot_name': prot_ids}), new_names], axis=1)
records = []
for record in SeqIO.parse("cleaned_assembly_proteins.fasta", "fasta"):
    id = record.id
    if id in list(new_names_dict['Prot_name']):
        record.description = record.description.replace(record.id, '')[1::]
        record.id = new_names_dict.loc[new_names_dict['Prot_name'] == id].iloc[0]['New_name']
        records.append(record)
SeqIO.write(records, "new_assembly_proteins.fasta", "fasta")

### Make new `.fasta` transcripts file

In [ ]:
data = pd.read_csv('merged.csv', delimiter=',')
ids = data.Name
new_names = pd.DataFrame({'New_name': ['Pmin_ref_seq_' + str(i+1) for i in range(len(data))]})
new_names_dict = pd.concat([pd.DataFrame({'Name': ids}), new_names], axis=1)
records = []
for record in SeqIO.parse("cleaned_assembly.fasta", "fasta"):
    id = record.id
    if id in list(new_names_dict['Name']):
        record.description = record.description.replace(record.id, '')[1::]
        record.id = new_names_dict.loc[new_names_dict['Name'] == id].iloc[0]['New_name']
        records.append(record)
SeqIO.write(records, "new_assembly.fasta", "fasta")

## Protein-coding sequences functional annotation

[EggNOG-mapper](http://eggnog-mapper.embl.de/) (v. 2.1.9) was used to annotate the obtained proteins. EggNOG database contains information about the orthologs of various organisms, the participation of the sequences in the biological processes, and the presence of certain protein domains.

## Protein-coding sequences differential expression analysis

RNentropy was utilised for the detection of significant variation of protein-coding sequences expression. The analysis was conducted using R (v. 4.2.3), the scripts here are only for female crabs.

In [ ]:
%%R

### Libraries ###
library(RNentropy)
library(dplyr)

### Input data ###
expr_unaveraged <- read.csv2("data/new_expression_data.csv", 
                             header = T, sep=",", row.names = 1)
expr_averaged <- read.csv2("data/new_mean_expression_data.csv", 
                           header = T, sep=",", row.names = 1)
colnames(expr_unaveraged) <- c("Im_12", "If_11", 
                               "Hm_2", "If_3", "Im_4", 
                               "Hm_10", "Hf_1")
Healthy_fem <- c(0, 0, 1)
Infected_fem <- c(1, 1, 0)
Sample <- c("If_11", "If_3", "Hf_1")  
design <- data.frame(Sample, Healthy_fem, Infected_fem)
design_matrix <- design
rownames(design_matrix) <- Sample
design_matrix <- design_matrix[, c(2, 3)]
species_tag <- "Pmin"

# Mutate #

expr_females <- expr_unaveraged %>% select(all_of(Sample)) 
expr_females <- mutate_all(expr_females, function(x) as.numeric(as.character(x)))
expr_females$GeneIDs <- as.factor(rownames(expr_females))
design_matrix <- data.matrix(design_matrix, rownames.force = T)

### Analysis ###
# compute statistics and p-values
RNresults <- RN_calc(expr_females, design = design_matrix)
# select only genes with significant changes of expression
# select sequences with global p-value lower than an user defined threshold 
# and provide a summary of over- and under-expression accoding to local p-values
RNresults_selected <- RN_select(RNresults, gpv_t = 0.01, 
                                lpv_t = 0.01, method = 'BH')
######
# NB! Был добавлен новый df - selected:
# Transcripts/genes with a corrected global p-value lower than gpv_t. 
# For each condition it will contain a column where values can be -1,0,1 or NA. 
# 1 means that all the replicates of this condition have expression value higher than the
# average and local p-value <= lpv_t (thus the corresponding gene will be over-expressed in this condition). 
# -1 means that all the replicates of this condition have expression value lower than the average and local p-value <= lpv_t (thus
# the corresponding gene will be under-expressed in this condition). 
# 0 means that at least one of the replicates has a local p-value > lpv_t. 
# NA means that the local p-values of the replicates are not consistent for this condition, that is, at least
# one replicate results to be over-expressed and at least one results to be under-expressed.
#######
RNresults_significant <- RNresults_selected$selected
write.table(RNresults_significant, 
            file=sprintf("%s_RNentropy_significant_results.tsv", species_tag),
            sep="\t", col.names = T, row.names = F)

### Subset over-expressed genes only ###
colnames(RNresults_significant)

for (sample in c("Healthy_fem", "Infected_fem")){
  sample_overexpr <- subset(RNresults_significant, 
                            RNresults_significant[[sample]] == "1")
  write.table(sample_overexpr, file=sprintf("%s_%s_RNentropy_overexp.tsv", 
                                            species_tag, sample),  sep="\t", 
              col.names = T, row.names = F)
}

### Compute point mutual information matrix for the experimental conditions ###
RNresults_pmi <- RN_pmi(RNresults)
RNresults_npmi_matrix <- RNresults_pmi$npmi
write.table(RNresults_npmi_matrix, file=sprintf("%s_RNentropy_npmi.tsv", 
                                                species_tag), sep="\t", 
            col.names = T, row.names = T)

## Gene Set Enrichment analysis (GSEA)

GeneOntology (GO) terms enrichment analysis was performed for the lists of differentially expressed protein-coding sequences. In present analysis, we selected only biological processes with p-value less than 0.01 and with more than 10 sequences. The analysis was also performed in R using topGO, rrvgo, dplyr packages and ggplot2, wordcloud, viridis packages for visualization. We utilised Drosophila melanogaster database as the closest avaliable relative to the anomuran Pagurus minutus (Arthropoda: Tetraconata) to simplify the redundance of GO sets by grouping similar terms based on their semantic similarity

Tre following scripts were used for female crabs wordcloud and topGO visualisations.

In [ ]:
%%R
### Libraries ###
library(wordcloud)
library(dplyr)
library(viridis)

### Input files ###
infected_reduced <- read.csv2("Pmin_Infected_fem_overexpr_genes_reduced_GSEA_results.Dmelanogaster.tsv", 
                             header = T, sep="\t")
healthy_reduced <- read.csv2("Pmin_Healthy_fem_overexpr_genes_reduced_GSEA_results.Dmelanogaster.tsv", 
                              header = T, sep="\t")
species_tag <- "Pmin"
set_tag <- "over-expressed_genes"

set.seed(1234)

### Processing ###
input_list <- list("infected_fem" = infected_reduced,
                   "healthy_fem" = healthy_reduced)

for (set in 1:length(input_list)){
  set_reduced_terms <- count(input_list[[set]], input_list[[set]]["parentTerm"], sort=T)
  colnames(set_reduced_terms) <- c("word", "freq")
  viridis_colors <- viridis(n = length(set_reduced_terms$word))
  # visual #
  pdf(file=sprintf("%s_%s_%s.wordcloud.pdf", species_tag, names(input_list[set]), set_tag), width = 9, height = 9)
  set_reduced_term_cloud <- wordcloud(words=set_reduced_terms$word, freq=set_reduced_terms$freq, min.freq = 1, 
                                      max.words=length(set_reduced_terms$word), scale = c(4.25, 0.25),
                                      random.order = F, rot.per = 0, colors = as.character(viridis_colors), ordered.colors = T)
  dev.off()
}

In [ ]:
%%R

### Libraries ###
library(topGO)
library(dplyr)
library(rrvgo)
library(ggplot2)

species_tag <- "Pmin"
options(ggrepel.max.overlaps = Inf)
gene_ont <- read.csv2("data/emapper_annotations.tsv", skip = 4,
                           header = T, sep="\t")
gene_ontuni <- gene_ont[, c("X.query", "GOs")]
gene_ontuni <- gene_ontuni[grep("GO", gene_ontuni$GOs),]
names(gene_ontuni) <- NULL

gene_ontuni <- as.data.frame(sapply(gene_ontuni, function(x)  as.character(x)))

write.table(gene_ontuni, file = sprintf("%s_GeneOntologyUniverse_filtered.txt", 
                                      species_tag), sep="\t",
            col.names = FALSE, row.names = FALSE)

### Input data ###
species_geneID2GO <- readMappings(file="data/Pmin_go.txt")
species_geneNames <- names(species_geneID2GO)

molsign_tab <- read.csv2("Pmin_RNentropy_significant_results.tsv", 
                         header = T, sep="\t")

stages <- colnames(molsign_tab[, 4:ncol(molsign_tab)])

### Analysis ###
for (stage in stages){
  stage_molsign <- dplyr::select(subset(molsign_tab, molsign_tab[stage] == 1), GeneIDs)
  stage_all_genes <- factor(as.integer(species_geneNames %in% stage_molsign$GeneIDs))
  names(stage_all_genes) <- species_geneNames
  ## Biological Processes (BP) ##
  GOdata_stage_BP <- new("topGOdata", ontology="BP", allGenes=stage_all_genes,
                         annot=annFUN.gene2GO, gene2GO=species_geneID2GO)
  resultsFisher_stage_BP <- runTest(GOdata_stage_BP, algorithm="classic", 
                                    statistic="fisher")
  resultsFisher_stage_BP_df <- as.data.frame(score(resultsFisher_stage_BP))
  colnames(resultsFisher_stage_BP_df) <- "P-values"
  resultsFisher_stage_BP_df_subset <- subset(resultsFisher_stage_BP_df, 
                                             resultsFisher_stage_BP_df$`P-values`
                                             < 0.01)
  results_stage_BP <- GenTable(GOdata_stage_BP, 
                               classicFisher=resultsFisher_stage_BP,
                               ranksOf="classicFisher", 
                               topNodes =length(resultsFisher_stage_BP_df_subset$`P-values`))
  results_stage_BP_short <- subset(results_stage_BP, Significant >= 10)
  output_file_name <- sprintf("%s_%s_overexpr_genes_GOenrichment_BP_Fisher.min_10_genes.tsv", 
                              species_tag, stage)
  write.table(results_stage_BP_short, file=output_file_name, sep="\t", quote=F, 
              col.names = T, row.names = F)
  ## Reduced Terms ##
  stage_simMatrix <- calculateSimMatrix(results_stage_BP_short[["GO.ID"]], 
                                        orgdb = "org.Dm.eg.db", 
                                        ont="BP", method="Rel")
  stage_classicFisher <- gsub("< ", "", results_stage_BP_short[["classicFisher"]])
  stage_scores <- setNames(-log10(as.numeric(as.character(stage_classicFisher))), 
                           results_stage_BP_short[["GO.ID"]])
  stage_reducedTerms <- reduceSimMatrix(stage_simMatrix, stage_scores, threshold = 0.7, 
                                        orgdb="org.Dm.eg.db")
  stage_reducedTerms_selected <- dplyr::select(stage_reducedTerms, parent, score, parentTerm)
  # duplication removing: #
  stage_reducedTerms_uniq <- stage_reducedTerms_selected %>% 
    group_by(parentTerm) %>%
    filter(row_number() == 1)
  # Preparation for visualization #
  stage_reducedTerms_uniq_df <- as.data.frame(stage_reducedTerms_uniq)
  stage_reducedTerms_uniq_df$GOid_and_desc <- paste(stage_reducedTerms_uniq_df$parent, 
                                                    "|", stage_reducedTerms_uniq_df$parentTerm)
  stage_reducedTerms_uniq_df$score_signif <- signif(stage_reducedTerms_uniq_df$score, 
                                                    digits = 3)
  # Visualization #
  # Plot #
  pdf(file=sprintf("%s_%s_overexpr_genes_reduced_GSEA_results.Dmelanogaster_parent_GOterms.pdf", species_tag, stage), width = 10, height = 12)
  result_plot <- stage_reducedTerms_uniq_df %>%
    arrange(score) %>%
    mutate(GOid_and_desc=factor(GOid_and_desc, levels=GOid_and_desc)) %>% 
    ggplot(aes(x=score, y=GOid_and_desc, fill=as.factor(score_signif))) + 
    geom_point(alpha=0.9, shape=21, size=5, color="black") + theme_bw() + 
    # legend.position = "right", axis.text=element_text(size=12, face="bold")
    theme(legend.position = "none", 
          plot.title = element_text(hjust = 0.5, size = 15, face="bold"), 
          axis.title.x = element_text(size=13, face="bold"), 
          axis.title.y = element_text(size=13, face="bold")) + 
    xlab("-log10(Fisher's Test p-values)") + ylab("Parental Gene Ontology terms (Fly)") + 
    ggtitle(sprintf("Reduced GSEA results: \n %s %s molecular signature \n (over-expressed genes)", 
                    species_tag, stage)) + labs(fill="Log-transformed scores")
  print(result_plot)
  dev.off()
  # ScatterPlot #
  pdf(file=sprintf("%s_%s_overexpr_genes_reduced_GSEA_results.Dmelanogaster.scatterPlot.pdf", 
                   species_tag, stage), width = 15, height = 15)
  scatter <- scatterPlot(stage_simMatrix, stage_reducedTerms, size="score", addLabel = TRUE, 
                         labelSize = 5)
  print(scatter)
  dev.off()
  # Output writing #
  write.table(stage_reducedTerms, 
              file=sprintf("%s_%s_overexpr_genes_reduced_GSEA_results.Dmelanogaster.tsv", 
                           species_tag, stage),
              sep="\t", quote = F, col.names = T, row.names = F)
}
# 